In [11]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from arcgis.features import FeatureLayer
from pathlib import Path
config = {"displayModeBar": False}
out_chart = Path(r"C:/Users/snewsome/Documents/GitHub/Monitoring/LTinfoUpdates")
template = 'plotly_white'
font     = 'Calibri'

# Set up file paths
# Gets data from the TRPA server
def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features
    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    # return data frame
    return all_data

def get_wildlife_data_web():
    wildlife_url = "https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/96"
    return get_fs_data(wildlife_url)

# plot bald eagle data
def plot_bald_eagle_winter(df, draft=False):
    # filter df for bald eagle
    df = df.loc[df['Wildlife_Species'] == 'Bald Eagle - winter']
    # add threshold value
    df['Threshold Value'] = 2

    # setup plot
    fig = px.scatter(df, x = 'Year', y= 'Total', color='Wildlife_Species')

    # update popup
    fig.update_traces(customdata=df['Wildlife_Species'],
                    hovertemplate='<b>%{y:.0f}</b> Bald Eagles Observed<extra></extra>',
                    name='Winter Bald Eagle Count',
                    marker=dict(color='#337ab7'))
    # # # create threshold line
    # fig.add_trace(go.Scatter(
    #     y=df['Threshold Value'],
    #     x=df['Year'],
    #     name= "Threshold",
    #     line=dict(color='#333333', width=2, dash='dash'),
    #     mode='lines',
    #     hovertemplate='Threshold: %{y:.0f}<extra></extra>'
    # ))
    # create trendline
    fig2 = px.scatter(df, x = 'Year', y= 'Total', 
                    trendline='ols', trendline_color_override='#8a7121')
    # set up trendline trace
    trendline = fig2.data[1]
    # get ols results
    fit_results = px.get_trendline_results(fig2).px_fit_results.iloc[0]
    # get beta value
    beta = fit_results.params[1]
    print("Beta = " + str(fit_results.params[1]))
    # add beta value from trend line to data frame
    df['Beta'] = fit_results.params[1]

    # create variable of beta
    slope = df['Beta']

    # update trendline legend and popup
    trendline.update(showlegend=True, name="Trend", line_width=3,
                    customdata=slope, 
                    hovertemplate='Trend: %{customdata:.2f}<extra></extra>')

    # add to figure
    fig.add_trace(trendline)

    # set layout
    fig.update_layout(title='Wintering Bald Eagle',
                        legend_title_text='',
                        font_family=font,
                        template=template,
                        hovermode="x unified",
                        showlegend=True,
                        dragmode=False,
                        xaxis = dict(
                            tickmode = 'linear',
                            dtick = 1
                        ),
                        yaxis = dict(
                            tickmode = 'linear',
                            tick0 = 0,
                            dtick = 5,
                            range=[0, 45],
                            title_text='Number of Eagles'
                        )       
                    )   
    fig.show()
   
    # export chart
   
    if draft == True:
        fig.write_html(
            config=config,
            file= out_chart / "Draft/Wildlife_BaldEagle_WinterLTinfo.html",
            include_plotlyjs="directory",
            div_id="Winter Bald Eagle",
            full_html=False,
            # default_height=500,
            # default_width=800,
        )   
    elif draft == False: 
        fig.write_html(
            config=config,
            file= out_chart / "Final/Wildlife_BaldEagle_WinterLTinfo.html",
            # include_plotlyjs="directory",
            div_id="Winter Bald_Eagle",
            full_html=False,
            # default_height=500,
            # default_width=800,
        )

df = get_wildlife_data_web()
plot_bald_eagle_winter(df, draft=True)
print("Bald Eagle Winter chart complete")



Beta = 0.8471794871794645


C:\Users\snewsome\AppData\Local\Temp\ipykernel_24216\4108361061.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\snewsome\AppData\Local\Temp\ipykernel_24216\4108361061.py:61: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



TypeError: write_html() missing 1 required positional argument: 'file'